
# ★ Systems Of Equations ★

In [6]:
# Import modules
import sys
import numpy as np
from scipy import linalg

# 2.1 Gaussian Elimination

In [5]:
def naive_gaussian_elimination(matrix):
    """
    A simple gaussian elimination to solve equations
    
    Args:
        matrix : numpy 2d array
        
    Returns:
        mat : The matrix processed by gaussian elimination
        x   : The roots of the equation
        
    Raises:
        ValueError:
            - matrix is null
        RuntimeError :
            - Zero pivot encountered
    """
    if matrix is None :
        raise ValueError('args matrix is null')
    
    #Clone the matrix
    mat = matrix.copy()
    
    # Row Size
    n = mat.shape[0]
    
    # Column Size
    m = mat.shape[1]
    
    # Gaussian Elimaination
    for j in xrange(0, n):
        if abs(mat[j , j]) == 0 :
            raise RuntimeError('zero pivot encountered')
        for i in xrange(j + 1, n):
            mult = mat[i, j] / mat[j, j]
            for k in xrange(j,m):
                mat[i, k] = mat[i, k] - mult * mat[j, k]
    
    # Back Substitution
    x = np.zeros(n, dtype=np.float)
    for i in xrange(n - 1,-1,-1):
        for j in xrange(i + 1, n):
            mat[i, m-1] = mat[i ,m-1] - mat[i,j] * x[j]
        x[i] = mat[i, m-1] / mat[i, i]
        
    return mat, x

### Example
Solve equations
\begin{matrix}
x + 2y - z = 3 & \\ 
2x + y - 2z = 3 & \\ 
-3x + y + z = -6 
\end{matrix}

In [6]:
"""
Input:
[[ 1  2 -1  3]
 [ 2  1 -2  3]
 [-3  1  1 -6]]
"""
input_mat = np.array([ 1, 2, -1, 3, 2, 1, -2, 3, -3, 1, 1, -6],dtype=np.float)
input_mat = input_mat.reshape(3, 4)
output_mat, x = naive_gaussian_elimination(input_mat)
print('----------------------')
print(output_mat)
print('----------------------')
print(x)

----------------------
[[ 1.  2. -1.  3.]
 [ 0. -3.  0. -3.]
 [ 0.  0. -2. -4.]]
----------------------
[ 3.  1.  2.]


# 2.2 The LU Factorization

### Example
Solve system
\begin{matrix}
x + 2y - z = 3 & \\ 
2x + y - 2z = 3 & \\ 
-3x + y + z = -6 
\end{matrix}
 
 using the LU factorization

In [11]:
A = np.array([ 1, 2, -1, 2, 1, -2, -3, 1, 1],dtype=np.float)
A = A.reshape(3, 3)
lu, piv = linalg.lu_factor(A)

b = np.array([3, 3, -6])
sol = linalg.lu_solve([lu, piv], b)
print(sol)

[ 3.  1.  2.]


### Example
Find the LU factorization of the given matrices. Check by matrix multiplication

$$
\begin{bmatrix}
3 & 1 & 2 \\ 
6 & 3 & 4 \\ 
3 & 1 & 5
\end{bmatrix}
$$

In [24]:
A = np.array([3, 1, 2, 6, 3, 4, 3, 1, 5], dtype=np.float).reshape(3,3)
P, L, U = linalg.lu(A)
print(A)
print('--------------------')
print(L)
print('--------------------')
print(U)
print('--------------------')
print(np.matmul(P,np.matmul(L, U)))

[[ 3.  1.  2.]
 [ 6.  3.  4.]
 [ 3.  1.  5.]]
--------------------
[[ 1.   0.   0. ]
 [ 0.5  1.   0. ]
 [ 0.5  1.   1. ]]
--------------------
[[ 6.   3.   4. ]
 [ 0.  -0.5  0. ]
 [ 0.   0.   3. ]]
--------------------
[[ 3.  1.  2.]
 [ 6.  3.  4.]
 [ 3.  1.  5.]]


### Example
Solve the system by LU factorization

$$
\begin{bmatrix}
3 & 1 & 2 \\ 
6 & 3 & 4 \\ 
3 & 1 & 5
\end{bmatrix}
\begin{bmatrix}
x_1 \\ 
x_2 \\ 
x_3
\end{bmatrix}
=
\begin{bmatrix}
0 \\ 
1 \\ 
3
\end{bmatrix}
$$

In [27]:
A = np.array([3, 1, 2, 6, 3, 4, 3, 1, 5], dtype=np.float).reshape(3,3)
b = np.array([0, 1, 3])
lu, piv = linalg.lu_factor(A)
x = linalg.lu_solve([lu, piv], b)
print(x)

[-1.  1.  1.]


# 2.4 The PA=LU Factorization

### Example
Apply Gaussian elimination with partial pivoting to solve the system

\begin{matrix}
  x_1 - x_2 + 3x_3 = -3 & \\ 
-1x_1 - 2x_3 = 1 & \\ 
 2x_1 + 2x_2 + 4x_3 = 0 
\end{matrix}

In [2]:
A = np.array([1, -1, 3, -1, 0, -2, 2, 2, 4]).reshape(3, 3)
b = np.array([-3, 1, 0])
lu, piv = linalg.lu_factor(A)
x = linalg.lu_solve([lu, piv], b)
print(x)

[ 1.  1. -1.]


### Example

Solve the system $2x_1 + 3x_2 = 4$,$3x_1 + 2x_2 = 1$ using the PA = LU factorization with partial pivoting

In [7]:
"""
[[2, 3]
 [3, 2]]
"""
A = np.array([2, 3, 3, 2]).reshape(2, 2)
b = np.array([4, 1])
lu, piv = linalg.lu_factor(A)
x = linalg.lu_solve([lu, piv], b)
print(x)

[-1.  2.]


# 2.5 Iterative Methods

## Jacobi Method

In [31]:
def jacobi_method(A, b, x0, k):
    """
    Use jacobi method to solve equations
    
    Args:
        A  (numpy 2d array): the matrix
        b  (numpy 1d array): the right hand side vector
        x0 (numpy 1d array): initial guess
        k  (real number): iterations
        
    Return:
        The approximate solution
        
    Exceptions:
        ValueError
            The size of matrix's column is not equal to the size of vector's size
    """
    if A.shape[1] is not x0.shape[0] :
        raise ValueError('The size of the columns of matrix A must be equal to the size of the x0')
    
    D = np.diag(A.diagonal())
    inv_D = linalg.inv(D) 
    LU = A - D
    xk = x0
    
    for _ in range(k):
        xk = np.matmul(b - np.matmul(LU, xk), inv_D)
    
    return xk

### Example
Apply the Jacobi Method to the system $3u + v = 5$, $u + 2v = 5$

In [34]:
A = np.array([3, 1, 1, 2]).reshape(2, 2)
b = np.array([5, 5])
x = jacobi_method(A, b, np.array([0, 0]), 20)
print('x = %s' %x)

x = [ 0.99999998  1.99999997]
